# Community Risk Index

## Setting the Scene

Crime is a complex, multi-dimensional phenomenon driven by economic hardship, social cohesion, physical environment and formal policing capacity. Rather than look at any single factor in isolation, your **Community Risk Index** brings together 25 variables—ranging from poverty rates to housing vacancy to police-per-capita—to create a holistic, data-driven ranking of neighbourhoods’ propensity for crime. This approach:

- **Reflects Social Disorganization Theory**, which links poverty, residential turnover, demographic heterogeneity and family structure to weakened informal social control;
- **Incorporates Routine Activity Theory**, which highlights how density, neighbourhood neglect and policing levels affect the “target–guardian” balance.

---

## Why this Problem?

- Crime not only inflicts economic and human costs, it exacerbates inequality and erodes trust in institutions.
- Policy-makers and community groups need a transparent, quantitative tool to pinpoint at-risk areas and the underlying drivers of that risk—so they can deploy resources where they’ll have greatest impact.

---

## Why this Data?

- The UCI **“Communities & Crime”** dataset contains 147 community-level attributes for 2,200+ U.S. jurisdictions, of which we select 25 high-quality variables covering all theoretical pillars.
- It’s a **single-file, publicly available CSV**—no arduous merging or licensing—and the variables directly map to the social and physical dimensions that criminology research (Shaw & McKay; Cohen & Felson) identifies as critical.

---

## Sub-Indices → Final Index

1. **Socio-economic Disadvantage**  
   (e.g. poverty rate, median income, unemployment)

2. **Residential Instability & Family Structure**  
   (e.g. residential turnover, two-parent family rates)

3. **Ethnic & Cultural Heterogeneity**  
   (racial composition shares)

4. **Housing & Density**  
   (owner-occupancy, vacancy, density, housing age)

5. **Policing Capacity**  
   (officers per capita, vehicles per capita)

Each set of variables is first combined into a **sub-index** (using equal weights and PCA for robustness), then the five sub-indices are **aggregated** into a single composite score. This workflow follows the **10-step OECD composite-indicator handbook** and aligns perfectly with your CA specification’s requirement to ground an index in theory, select appropriate data, and construct transparent sub-indices before forming the final measure.